# Práctica 1 – Simulación de cartera de Autos

En este cuaderno construiremos un dataset sintético para representar una cartera de pólizas de autos. Asumimos tres coberturas principales: **Daños Materiales (DM)**, **Robo Total (Robo)** y **Responsabilidad Civil (RC)**. Los parámetros de frecuencia y severidad se basan en supuestos ficticios que pueden ser ajustados.

## Importación de librerías y fijación de semilla

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Fijamos semilla para reproducibilidad
np.random.seed(2025)

## Definición de parámetros y segmentos

In [2]:
# Número de pólizas a simular
n_polizas = 10000

# Frecuencias base por cobertura (probabilidad de siniestro por año)
freq_base = {"DM": 0.05, "Robo": 0.04, "RC": 0.06}

# Severidades medias por cobertura (en pesos)
sev_media = {"DM": 21000, "Robo": 15000, "RC": 24000}

# Parámetros de la lognormal: elegimos una desviación logarítmica moderada
sigma_ln = 0.6
mu_ln = {k: np.log(v) - 0.5 * sigma_ln**2 for k, v in sev_media.items()}

# Definición de segmentos y multiplicadores de frecuencia
segmentos = pd.DataFrame({
    'segmento': ['Bajo riesgo', 'Estándar', 'Alto riesgo'],
    'prob_segmento': [0.3, 0.5, 0.2],
    'mult_freq': [0.7, 1.0, 1.5]
})
segmentos

,segmento,prob_segmento,mult_freq
0,Bajo riesgo,0.3,0.7
1,Estándar,0.5,1.0
2,Alto riesgo,0.2,1.5


## Simulación de pólizas y siniestros

In [3]:
# Asignamos segmento a cada póliza
polizas = pd.DataFrame({
    'id_poliza': np.arange(1, n_polizas+1),
    'segmento': np.random.choice(segmentos['segmento'], size=n_polizas, p=segmentos['prob_segmento'])
})

# Añadimos multiplicador de frecuencia
polizas = polizas.merge(segmentos[['segmento', 'mult_freq']], on='segmento', how='left')

# Función para simular siniestros y severidades por cobertura
def simular_cobertura(freq_base, mult_freq, mu, sigma, size):
    lam = freq_base * mult_freq
    # Número de siniestros ~ Poisson(lam)
    n_siniestros = np.random.poisson(lam, size=size)
    severidades = []
    for n in n_siniestros:
        if n == 0:
            severidades.append(0.0)
        else:
            # Suma de siniestros lognormales
            severidades.append(np.sum(np.random.lognormal(mean=mu, sigma=sigma, size=n)))
    return np.array(n_siniestros), np.array(severidades)

# Simulamos para cada cobertura
n_siniestros_DM, siniestro_DM = simular_cobertura(freq_base['DM'], polizas['mult_freq'], mu_ln['DM'], sigma_ln, n_polizas)
n_siniestros_Robo, siniestro_Robo = simular_cobertura(freq_base['Robo'], polizas['mult_freq'], mu_ln['Robo'], sigma_ln, n_polizas)
n_siniestros_RC, siniestro_RC = simular_cobertura(freq_base['RC'], polizas['mult_freq'], mu_ln['RC'], sigma_ln, n_polizas)

# Incorporamos a la cartera
cartera = polizas.copy()
cartera['n_siniestros_DM'] = n_siniestros_DM
cartera['siniestro_DM'] = siniestro_DM
cartera['n_siniestros_Robo'] = n_siniestros_Robo
cartera['siniestro_Robo'] = siniestro_Robo
cartera['n_siniestros_RC'] = n_siniestros_RC
cartera['siniestro_RC'] = siniestro_RC
cartera['siniestro_total'] = cartera[['siniestro_DM','siniestro_Robo','siniestro_RC']].sum(axis=1)
cartera.head()

,id_poliza,segmento,mult_freq,n_siniestros_DM,siniestro_DM,n_siniestros_Robo,siniestro_Robo,n_siniestros_RC,siniestro_RC,siniestro_total
0,1,Bajo riesgo,0.7,0,0.0,0,0.0,0,0.0,0.0
1,2,Alto riesgo,1.5,0,0.0,0,0.0,0,0.0,0.0
2,3,Alto riesgo,1.5,0,0.0,0,0.0,0,0.0,0.0
3,4,Estándar,1.0,0,0.0,0,0.0,0,0.0,0.0
4,5,Estándar,1.0,0,0.0,0,0.0,0,0.0,0.0


## Cálculo de primas y razón de siniestralidad

In [4]:
# Prima pura por cobertura (frecuencia x severidad media) por póliza
prima_DM_pura = freq_base['DM']   * cartera['mult_freq'] * sev_media['DM']
prima_Robo_pura = freq_base['Robo'] * cartera['mult_freq'] * sev_media['Robo']
prima_RC_pura = freq_base['RC']   * cartera['mult_freq'] * sev_media['RC']

# Aplicamos margen de gastos (30%)
margin = 0.3
cartera['prima_DM_comercial'] = prima_DM_pura * (1 + margin)
cartera['prima_Robo_comercial'] = prima_Robo_pura * (1 + margin)
cartera['prima_RC_comercial'] = prima_RC_pura * (1 + margin)

cartera['prima_total'] = cartera[['prima_DM_comercial', 'prima_Robo_comercial', 'prima_RC_comercial']].sum(axis=1)

# Razón de siniestralidad (loss ratio) por póliza
cartera['LR'] = np.where(cartera['prima_total'] > 0, cartera['siniestro_total'] / cartera['prima_total'], np.nan)

# Resumen por segmento
resumen = cartera.groupby('segmento').agg({
    'id_poliza': 'count',
    'siniestro_total': 'sum',
    'prima_total': 'sum'
})
resumen.rename(columns={'id_poliza': 'polizas'}, inplace=True)
resumen['LR_promedio'] = resumen['siniestro_total'] / resumen['prima_total']
resumen

,polizas,siniestro_total,prima_total,LR_promedio
segmento,,,,
Alto riesgo,1988,9.402457e+06,11978694.0,0.784932
Bajo riesgo,2939,6.343404e+06,8264174.1,0.767579
Estándar,5073,1.581335e+07,20378241.0,0.775992


## Exportación del dataset

In [5]:
# Guardamos la cartera simulada en un archivo CSV
cartera.to_csv('cartera_autos_simulada_Python.csv', index=False)
cartera.head()

,id_poliza,segmento,mult_freq,n_siniestros_DM,siniestro_DM,n_siniestros_Robo,siniestro_Robo,n_siniestros_RC,siniestro_RC,siniestro_total,prima_DM_comercial,prima_Robo_comercial,prima_RC_comercial,prima_total,LR
0,1,Bajo riesgo,0.7,0,0.0,0,0.0,0,0.0,0.0,955.5,546.0,1310.4,2811.9,0.0
1,2,Alto riesgo,1.5,0,0.0,0,0.0,0,0.0,0.0,2047.5,1170.0,2808.0,6025.5,0.0
2,3,Alto riesgo,1.5,0,0.0,0,0.0,0,0.0,0.0,2047.5,1170.0,2808.0,6025.5,0.0
3,4,Estándar,1.0,0,0.0,0,0.0,0,0.0,0.0,1365.0,780.0,1872.0,4017.0,0.0
4,5,Estándar,1.0,0,0.0,0,0.0,0,0.0,0.0,1365.0,780.0,1872.0,4017.0,0.0


## Reflexión final

El ejercicio muestra cómo a partir de unos pocos supuestos de frecuencia y severidad, ajustados por segmento, se puede generar un dataset rico en información que ayude a analizar la siniestralidad y las primas esperadas de un producto. Recuerda contrastar las métricas obtenidas con los indicadores publicados por la CNSF y documentar los supuestos adoptados.